In [ ]:
# %% [markdown]
# # Quran Verse Finder - Backend Development
# This notebook will help us develop and test the backend for our Quran verse finder app

# %%
# Import necessary libraries
import json
import os
import requests
from flask import Flask, jsonify, request
from flask_cors import CORS

# %% [markdown]
# ## Download Quran Data
# First, let's download the Quran data (focusing on the 30th part)

# %%
def download_quran_data():
    """Download Quran data if it doesn't exist locally"""
    if not os.path.exists('quran_data.json'):
        print("Downloading Quran data...")
        # Using the Quran.com API to get the data
        response = requests.get('https://api.quran.com/api/v4/quran/verses/uthmani')
        
        if response.status_code == 200:
            all_verses = response.json()['verses']
            
            # Filter for Juz' 30 (from Surah 78 to 114)
            juz_30_start_verse = 5673  # Approximate starting verse of Juz 30
            juz_30_verses = all_verses[juz_30_start_verse:]
            
            with open('quran_data.json', 'w', encoding='utf-8') as f:
                json.dump(juz_30_verses, f, ensure_ascii=False, indent=2)
            print("Quran data downloaded successfully!")
        else:
            print(f"Failed to download Quran data. Status code: {response.status_code}")
    else:
        print("Quran data already exists locally.")

# %%
# Execute the download function
download_quran_data()

# %% [markdown]
# ## Create helper functions to search verses

# %%
def load_quran_data():
    """Load the Quran data from the local file"""
    with open('quran_data.json', 'r', encoding='utf-8') as f:
        return json.load(f)

# %%
def find_verses_by_letter_position(letter, position, limit=5):
    """
    Find verses containing the specified Arabic letter in the specified position
    
    Parameters:
    - letter: Arabic letter to search for
    - position: 'first', 'middle', or 'last'
    - limit: Maximum number of verses to return
    
    Returns:
    - List of matching verses with their metadata
    """
    verses = load_quran_data()
    matching_verses = []
    
    for verse in verses:
        # Get the text content of the verse
        verse_text = verse['text_uthmani']
        words = verse_text.split()
        
        found = False
        if position == 'first':
            # Check if any word starts with the letter
            for word in words:
                if word.startswith(letter):
                    found = True
                    break
        elif position == 'last':
            # Check if any word ends with the letter
            for word in words:
                if word.endswith(letter):
                    found = True
                    break
        elif position == 'middle':
            # Check if the letter appears in the middle of any word
            for word in words:
                if len(word) > 2 and letter in word[1:-1]:
                    found = True
                    break
        
        if found:
            # Add verse to results
            matching_verses.append({
                'id': verse['id'],
                'surah': verse['surah_number'],
                'verse_number': verse['verse_number'],
                'text': verse['text_uthmani'],
                'audio_url': f"https://verses.quran.com/{verse['surah_number']}/{verse['verse_number']}.mp3"
            })
            
            if len(matching_verses) >= limit:
                break
    
    return matching_verses

# %% [markdown]
# ## Test our search function

# %%
# Let's test with the letter 'ا' (alif) in the first position
test_results = find_verses_by_letter_position('ا', 'first', 3)
print(f"Found {len(test_results)} verses")
for verse in test_results:
    print(f"Surah {verse['surah']}, Verse {verse['verse_number']}")
    print(verse['text'])
    print("---")

# %% [markdown]
# ## Set up Flask API

# %%
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests from React

@app.route('/api/search', methods=['GET'])
def search_verses():
    letter = request.args.get('letter', '')
    position = request.args.get('position', 'first')
    limit = int(request.args.get('limit', 5))
    
    if not letter:
        return jsonify({'error': 'Letter parameter is required'}), 400
    
    verses = find_verses_by_letter_position(letter, position, limit)
    return jsonify({'verses': verses})

@app.route('/api/letters', methods=['GET'])
def get_arabic_letters():
    # Arabic alphabet
    arabic_letters = ['ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 
                      'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 
                      'ن', 'ه', 'و', 'ي']
    return jsonify({'letters': arabic_letters})

# %% [markdown]
# ## Run the Flask server (for testing in the notebook)

# %%
# Uncomment to run the server (this will block the notebook execution)
# if __name__ == '__main__':
#     app.run(debug=True, port=5000)

# %% [markdown]
# ## Create a function to start the server from outside the notebook

# %%
def start_server():
    app.run(debug=True, port=5000)

print("Backend functions are ready. Run start_server() to launch the API server.")